In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir(os.path.join(os.getcwd(),".."))

In [3]:
import numpy as np
import scipy as sp
from pathlib import Path

from utils.functions import * 
from scipy import signal
from scipy.io import wavfile
from pydub import AudioSegment
from matplotlib import pyplot as plt
%matplotlib widget
import IPython.display as ipd

In [4]:
def create_sinusoid(rate, f_s, s_m, duration=1):
    """
    
    :param rate: sampling rate. Usually 44100
    :param f_s: signal frequency
    :param s_m: signal magnitude (usually voltage)
    :param duration: duration in seconds
    :return: sinusoid signal
    """
    time = np.linspace(0,duration, num=rate*duration)
    s = s_m*np.sin(2*np.pi*f_s*time)
#     w = hann(rate*duration)
#     y = s*w
    return s

# Αρχικοποίηση φακέλων για τα αρχεία εισόδου και τα παραγόμενα αρχεία 

In [5]:
source_audio_dir = Path("source_audio_files")
generated_audio_files_python = Path("generated_audio_files_python").joinpath("askisi4")
generated_audio_files_python.mkdir(exist_ok=True, parents=True)
generated_audio_files_ffmpeg = Path("generated_audio_files_ffmpeg").joinpath("askisi4")
generated_audio_files_matlab = Path("generated_audio_files_matlab").joinpath("askisi4")
generated_audio_files_audacity = Path("generated_audio_files_audacity").joinpath("askisi4")

# Μέρος 1: Δημιουργία ημιτονικού σήματος

Δημιουργία και κβάντιση ημιτονικού σήματος

In [6]:
rate = 44100
duration = 5
s1 = create_sinusoid(rate,5000,0.98,duration)
y1 = quantise_audio(s1,16,1,-1,0,1)

In [7]:
dt = 1/rate
npts = rate*5
ft_sin = np.fft.rfft(y1)
ps_sin = np.real(ft_sin*np.conj(ft_sin))*np.square(dt)
w = np.fft.fftfreq(npts, d=dt)
w = np.abs(w[:npts//2+1]) # only freqs for real fft
dBsin = 20*np.log10(ps_sin/np.max(ps_sin))

In [8]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
ax.semilogx(w, ps_sin)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
ax = plotSpectrum(y1, 44100, db=True, logx=True, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
f2, Pn = sp.signal.periodogram(y1)

In [11]:
ax.semilogy(f2,Pn)

/home/user/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  """Entry point for launching an IPython kernel.


In [12]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
ax = plotSpectrum(y1, 44100, db=True, logx=True, ax=ax)
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('Magnitude (dB)')
# ax.set_ylim([-200,5])
ax.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
wavfile.write(generated_audio_files_python.joinpath("sine.wav"),44100, y1)

## Κωδικοποίηση του ημιτονικού σήματος σε μορφή mp4

Για την κωδικοποίηση χρησιμοποιείται η βιβλιοθήκη της python [`pydub`](https://github.com/jiaaro/pydub) καθώς η συνάρτηση `wavfile.write` δεν υποστηρίζει την εγγραφή με την συγκεκριμένη κωδικοποίηση όπως η αντίστοιχη συνάρτηση `audiowrite` της matlab.

Για την δημιουργία του mp4 αρχείου χρησιμοποιείται η εντολή  
```
ffmpeg -i sine.wav -b:a 96k sine.mp4
```
η οποία χρησιμοποιεί τα codecs της βιβλιοθήκης [`ffmpeg`](https://www.ffmpeg.org/) η οποία πρέπει να είναι εγκατεστημένη στο σύστημά μας.

In [14]:
# audio_segment = AudioSegment(
#     y1.astype(np.float32).tobytes(), 
#     frame_rate=44100,
#     sample_width=y1.dtype.itemsize, 
#     channels=1
# )

In [15]:
# audio_segment.export("sine.mp4", format="mp4", bitrate="96k")

Εισαγωγή του αρχείου μορφής mp4 και αποκωδικοποίηση του σε `wav`.

In [16]:
sine_mp4 = AudioSegment.from_file(Path(generated_audio_files_ffmpeg / "sine.mp4"), format="mp4")
sine_mp4.export(generated_audio_files_python.joinpath("sinedcc.wav"), format="wav")
rate, sinedcc = wavfile.read(Path(generated_audio_files_python / "sinedcc.wav"))

Εισαγωγή του αρχείου που προέκυψε από την κωδικοποίηση/αποκωδικοποίηση με χρήση του Audacity

In [17]:
rate, sinedc = wavfile.read(Path(generated_audio_files_audacity / "sinedc.wav"))

Εισαγωγή του αρχικού αρχείου `sine.wav` που δημιουργήθηκε από την numpy.

In [18]:
rate, sine = wavfile.read(Path(generated_audio_files_python / "sine.wav"))

In [19]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
ax = plotSpectrum(sine, 44100, db=True, logx=True, ax=ax)
ax = plotSpectrum(sinedc, 44100, db=True, logx=True, ax=ax)
ax = plotSpectrum(sinedcc, 44100, db=True, logx=True, ax=ax)
ax.legend(['wav quality','audacity quality','ffmpeg quality'])
ax.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Μέρος 2: Εφαρμογή της διαδικασίας στα αρχεία bongo.wav και xylophone.wav

### Αρχείο bongo.wav

Εισαγωγή του αρχείου μορφής mp4 και αποκωδικοποίηση του σε `wav`.

In [20]:
bongo_mp4 = AudioSegment.from_file(Path(generated_audio_files_ffmpeg / "bongo.mp4"), format="mp4")
bongo_mp4.export("bongodcc.wav", format="wav")
rate, bongodcc = wavfile.read("bongodcc.wav")

Εισαγωγή του αρχείου που προέκυψε από την κωδικοποίηση/αποκωδικοποίηση με χρήση του Audacity

In [21]:
rate, bongodc = wavfile.read(Path(generated_audio_files_audacity / "bongodc.wav"))

Εισαγωγή του αρχικού αρχείου `bongo.wav`.

In [22]:
rate, bongo = wavfile.read(Path(source_audio_dir / "bongo.wav"))

/home/user/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: WavFileWarning: Chunk (non-data) not understood, skipping it.
  """Entry point for launching an IPython kernel.


In [23]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
ax = plotSpectrum(bongo, 44100, db=True, logx=True, ax=ax)
ax = plotSpectrum(bongodc, 44100, db=True, logx=True, ax=ax)
ax = plotSpectrum(bongodcc, 44100, db=True, logx=True, ax=ax)
ax.legend(['wav quality','audacity quality','ffmpeg quality'])
ax.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Αρχείο xylophone.wav

Εισαγωγή του αρχείου μορφής mp4 και αποκωδικοποίηση του σε `wav`.

In [24]:
xylophone_mp4 = AudioSegment.from_file(Path(generated_audio_files_ffmpeg / "xylophone.mp4"), format="mp4")
xylophone_mp4.export("xylophonedcc.wav", format="wav")
rate, xylophonedcc = wavfile.read("xylophonedcc.wav")

Εισαγωγή του αρχείου που προέκυψε από την κωδικοποίηση/αποκωδικοποίηση με χρήση του Audacity

In [25]:
rate, xylophonedc = wavfile.read(Path(generated_audio_files_audacity / "xylophonedc.wav"))

Εισαγωγή του αρχικού αρχείου `xylophone.wav`.

In [26]:
rate, xylophone = wavfile.read(source_audio_dir / "xylophone.wav")

/home/user/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: WavFileWarning: Chunk (non-data) not understood, skipping it.
  """Entry point for launching an IPython kernel.


In [27]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
ax = plotSpectrum(xylophone, 44100, db=True, logx=True, ax=ax)
ax = plotSpectrum(xylophonedc, 44100, db=True, logx=True, ax=ax)
ax = plotSpectrum(xylophonedcc, 44100, db=True, logx=True, ax=ax)
ax.legend(['wav quality','audacity quality','ffmpeg quality'])
ax.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

##  Συγχρονισμός των αρχείων ήχου για το ημιτονικό σήμα 

In [28]:
rate, sinedccm = wavfile.read(Path(generated_audio_files_matlab / "sinedccm.wav"))

In [29]:
fig2, ax = plt.subplots(1, 1, figsize=(8, 6))
ax.cla()
ax.plot(sine[:50])
ax.plot(sinedc[:50])
ax.plot(sinedcc[:50]/32767)
ax.plot(sinedccm[:50]*32767)
ax.legend(['orig', 'mp3', 'ffmpeg', 'matlab'])
ax.set_title("Samples of the first 50 samples of sinusoids\ngenerated in different tools")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Samples of the first 50 samples of sinusoids\ngenerated in different tools')

Το ερώτημα για το time alignment των ημιτόνων δεν υλοποιήθηκε, καθώς δεν βρέθηκε βιβλιοθήκη που να υλοποιεί την `alignsignals` συνάρτηση της MATLAB σε python. Ένα παράδειγμα που θα επιτρέψει την υλοποίηση του ερωτήματος ωστόσο με τη χρήση της συνάρτησης `np.correlate` υπάρχει στον [σύνδεσμο](https://biosignalsplux.com/learn/notebooks/Categories/Pre-Process/synchronisation_rev.php).